In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = trp
band = [5]
commitnumber = de5d43e
molecule = {'co2': 0.0004}
tsfc = 300


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o co2 band3c
  atmpro = mls
  band = 3c
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.375563e-24
  molecule = co2
  ng_adju = [0, 0, 0]
  ng_refs = [3, 2, 4]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (10, 250), (500, 250)]
  tsfc = 294
  vmax = 800
  vmin = 720
  w_diffuse = [(1.7, 1.6, 1.8), (1.8, 1.7), (1.5, 1.6, 1.7, 1.8)]
  wgt = [(0.6, 0.4, 0.7), (0.7, 0.4), (0.3, 0.4, 0.85, 0.9)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0  0.253465                  NaN       NaN
0.000312      NaN       NaN                  1.0  0.099271
0.000750      2.0  0.253537                  2.0  0.103148
0.001052      3.0  0.264369                  3.0  0.112938
0.001476      4.0  0.274174                  4.0  0.122779
0.002070      5.0  0.282045                  5.0  0.132421
0.002904      6.0  0.287591                  6.0  0.141618
0.004074      7.0  0.289838                  7.0  0.150001
0.005714      8.0  0.287958                  8.0  0.157184
0.008015      9.0  0.280797                  9.0  0.162394
0.011243     10.0  0.267112                 10.0  0.164789
0.015771     11.0  0.245594                 11.0  0.162998
0.022122     12.0  0.214727                 12.0  0.155088
0.031031     13.0  0.172619                 13.0  0.138633
0.043528     14.0  0.120194                 14.0  0.113665
0.061057     15.0  0.101197                 15.0  0.115928
0.085645     16.0  0.138739                 16.0  0.167010
0.120136     17.0  0.189009                 17.0  0.227803
0.168516     18.0  0.226301                 18.0  0.263596
0.236378     19.0  0.257389                 19.0  0.270968
0.331549     20.0  0.294454                 20.0  0.270841
0.465100     21.0  0.360340                 21.0  0.311395
0.652400     22.0  0.474476                 22.0  0.431936
0.915100     23.0  0.481580                 23.0  0.464317
1.283650     24.0  0.401915                 24.0  0.407057
1.800600     25.0  0.337105                 25.0  0.349678
2.525700     26.0  0.266892                 26.0  0.279376
3.542800     27.0  0.212055                 27.0  0.228196
4.969550     28.0  0.170304                 28.0  0.190892
6.970850     29.0  0.136095                 29.0  0.158628
9.778100     30.0  0.106924                 30.0  0.123854
13.715850    31.0  0.081764                 31.0  0.089746
19.239350    32.0  0.060297                 32.0  0.059611
26.987250    33.0  0.042726                 33.0  0.036269
37.855300    34.0  0.029791                 34.0  0.022821
53.100050    35.0  0.011939                 35.0  0.008260
73.887500    36.0 -0.008907                 36.0 -0.010590
97.662500    37.0 -0.022268                 37.0 -0.024255
121.437500   38.0 -0.021291                 38.0 -0.023920
145.212500   39.0 -0.011096                 39.0 -0.014454
168.987500   40.0 -0.005018                 40.0 -0.008480
192.762500   41.0  0.000938                 41.0 -0.002124
216.537500   42.0  0.007116                 42.0  0.004203
240.312500   43.0  0.013505                 43.0  0.010396
264.087500   44.0  0.020033                 44.0  0.016307
287.862500   45.0  0.026625                 45.0  0.022029
311.637500   46.0  0.033208                 46.0  0.027604
335.412500   47.0  0.039720                 47.0  0.033117
359.187500   48.0  0.046111                 48.0  0.038692
382.962500   49.0  0.052332                 49.0  0.044334
406.737500   50.0  0.058358                 50.0  0.050088
430.512500   51.0  0.064184                 51.0  0.056015
454.287500   52.0  0.069804                 52.0  0.062131
478.062500   53.0  0.075213                 53.0  0.068368
501.837500   54.0  0.080427                 54.0  0.074742
525.612500   55.0  0.085477                 55.0  0.081089
549.387500   56.0  0.090398                 56.0  0.087563
573.162500   57.0  0.095256                 57.0  0.094085
596.937500   58.0  0.100152                 58.0  0.100668
620.712500   59.0  0.105185                 59.0  0.107365
644.487500   60.0  0.110484                 60.0  0.114215
668.262500   61.0  0.115853                 61.0  0.120999
692.037500   62.0  0.120184                 62.0  0.126590
715.812500   63.0  0.121458                 63.0  0.128924
739.587500   64.0  0.118810                 64.0  0.127211
76

# Fluxes by Layer

CRD                           CLIRAD             \
                         flug       fldg      fnetg       flug       fldg   
pressure     level                                                          
0.000000e+00 1     -27.530084   0.000000 -27.530084        NaN        NaN   
1.000000e-08 1            NaN        NaN        NaN -27.161709   0.000001   
6.244000e-04 2     -27.530099   0.000034 -27.530065 -27.161725   0.000025   
8.759000e-04 3     -27.530106   0.000048 -27.530058 -27.161732   0.000035   
1.228600e-03 4     -27.530114   0.000068 -27.530047 -27.161742   0.000049   
1.723400e-03 5     -27.530126   0.000095 -27.530031 -27.161755   0.000070   
2.417400e-03 6     -27.530142   0.000135 -27.530007 -27.161773   0.000099   
3.390900e-03 7     -27.530165   0.000191 -27.529974 -27.161798   0.000140   
4.756500e-03 8     -27.530196   0.000269 -27.529927 -27.161833   0.000200   
6.672000e-03 9     -27.530240   0.000378 -27.529862 -27.161883   0.000285   
9.358900e-03 10    -27.530301   0.000528 -27.529773 -27.161951   0.000405   
1.312780e-02 11    -27.530386   0.000732 -27.529653 -27.162048   0.000576   
1.841450e-02 12    -27.530506   0.001007 -27.529500 -27.162185   0.000815   
2.583020e-02 13    -27.530679   0.001368 -27.529311 -27.162381   0.001147   
3.623230e-02 14    -27.530932   0.001834 -27.529098 -27.162665   0.001602   
5.082340e-02 15    -27.531317   0.002427 -27.528891 -27.163085   0.002218   
7.129060e-02 16    -27.531889   0.003244 -27.528645 -27.163690   0.003104   
1.000000e-01 17    -27.532682   0.004509 -27.528173 -27.164515   0.004498   
1.402710e-01 18    -27.533725   0.006454 -27.527272 -27.165586   0.006655   
1.967600e-01 19    -27.535053   0.009296 -27.525757 -27.166924   0.009757   
2.759970e-01 20    -27.536662   0.013321 -27.523341 -27.168516   0.013892   
3.871000e-01 21    -27.538432   0.018966 -27.519466 -27.170232   0.019173   
5.431000e-01 22    -27.539813   0.027006 -27.512807 -27.171528   0.026224   
7.617000e-01 23    -27.538962   0.038441 -27.500521 -27.170464   0.036347   
1.068500e+00 24    -27.535146   0.052127 -27.483019 -27.166266   0.049024   
1.498800e+00 25    -27.529373   0.066840 -27.462533 -27.159930   0.063439   
2.102400e+00 26    -27.521780   0.083351 -27.438429 -27.151526   0.080040   
2.949000e+00 27    -27.513109   0.101445 -27.411664 -27.141730   0.098265   
4.136600e+00 28    -27.503570   0.121738 -27.381832 -27.130589   0.119230   
5.802500e+00 29    -27.493115   0.144890 -27.348225 -27.117911   0.144226   
8.139200e+00 30    -27.481810   0.171256 -27.310554 -27.103500   0.173727   
1.141700e+01 31    -27.469919   0.200881 -27.269037 -27.087552   0.205874   
1.601470e+01 32    -27.458007   0.233500 -27.224506 -27.070670   0.237875   
2.246400e+01 33    -27.447078   0.268637 -27.178442 -27.054248   0.266999   
3.151050e+01 34    -27.438483   0.305828 -27.132655 -27.040258   0.291879   
4.420010e+01 35    -27.432160   0.344286 -27.087874 -27.028871   0.314800   
6.200000e+01 36    -27.437790   0.375089 -27.062700 -27.032214   0.335561   
8.577500e+01 37    -27.479810   0.392024 -27.087787 -27.079712   0.353230   
1.095500e+02 38    -27.559565   0.409065 -27.150500 -27.171094   0.376294   
1.333250e+02 39    -27.671250   0.460787 -27.210463 -27.297452   0.435279   
1.571000e+02 40    -27.802310   0.560598 -27.241712 -27.443646   0.540761   
1.808750e+02 41    -27.950929   0.695083 -27.255846 -27.606993   0.680223   
2.046500e+02 42    -28.115502   0.862297 -27.253204 -27.786306   0.853552   
2.284250e+02 43    -28.294505   1.061342 -27.233164 -27.981303   1.060387   
2.522000e+02 44    -28.486391   1.291260 -27.195130 -28.189925   1.298290   
2.759750e+02 45    -28.689781   1.551071 -27.138710 -28.410334   1.564629   
2.997500e+02 46    -28.903397   1.839673 -27.063724 -28.641025   1.857367   
3.235250e+02 47    -29.126040   2.155839 -26.970201 -28.880463   2.174555   
3.473000e+02 48    -29.356586   2.498247 -26.858338 -29.127209   2.514577   
3.710750e+02 49    -29

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')